In [1]:
#Install necessary libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns7

In [2]:
customers_data = pd.read_csv(r'../Data/addresses.csv')
customers_data.head()

,company_id,address,total_spend
0,1,"APARTMENT 2,\n52 BEDFORD ROAD,\nLONDON,\nENGLA...",5700
1,2,"107 SHERINGHAM AVENUE,\nLONDON,\nN14 4UJ",4700
2,3,"43 SUNNINGDALE,\nYATE,\nBRISTOL,\nENGLAND,\nBS...",5900
3,4,"HAWESWATER HOUSE,\nLINGLEY MERE BUSINESS PARK,...",7200
4,5,"AMBERFIELD BARN HOUSE AMBER LANE,\nCHART SUTTO...",4600


### Data Overview and Documentation

When working with new datasets, it's crucial to document the structure and content of the data for several reasons:
1. Ensures clarity for team members and stakeholders
2. Facilitates future data processing and analysis
3. Provides a foundation for database schema design
4. Aids in maintaining data governance standards

### Customer Data Dictionary

| Column Name | Data Type | Description |
|-------------|-----------|-------------|
| company_id  | INT64     | Unique identifier assigned to each customer company |
| address     | STRING    | Complete postal address of the customer |
| total_spend | INT64     | Cumulative spending amount per customer (in GBP) |

**Note**: This dataset contains transaction records for business customers, with monetary values represented in British Pounds Sterling (GBP).

In [3]:
customers_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   company_id   100000 non-null  int64 
 1   address      99032 non-null   object
 2   total_spend  100000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 2.3+ MB


In [4]:
customers_data.shape

(100000, 3)

### Data Exploration Insights

Understanding the fundamental characteristics of our dataset (such as shape and info ) provides several key advantages:

1. **Data Structure Analysis**
   - Identification of data types per column
   - Assessment of storage requirements
   - Overview of dataset dimensions and scale

2. **Query Optimization**
   - Informs decisions about data partitioning strategies
   - Helps optimize cloud computing costs, particularly in platforms like Google BigQuery
   - Enables efficient query planning and execution

3. **Performance Considerations**
   - Guides decisions on indexing strategies
   - Helps determine appropriate storage optimization techniques
   - Facilitates cost-effective query execution planning

This preliminary analysis is crucial for both data engineering decisions and cost optimization in cloud environments.

In [5]:
customers_data['total_spend'].describe()

count    100000.000000
mean       4951.662000
std        1500.983866
min           0.000000
25%        3900.000000
50%        5000.000000
75%        6000.000000
max       11700.000000
Name: total_spend, dtype: float64

In [6]:
#After understanding the data, we need to investigate if there are any missing values or duplicates.
customers_data.isnull().sum()

company_id       0
address        968
total_spend      0
dtype: int64

We can see there are 968 missing addresses, which is just under 1% of our rows. Since we have no way of knowing the addresses of those missing customers just from the data provided, we can safely drop these rows:

In [7]:
customers_data = customers_data.dropna(subset=['address'])
customers_data.shape


(99032, 3)

Losing 1% due to missing key information is fine. If, say, 10% of our customers had missing addresses, we might want to examine why. Alternative solution is, categorizing missing addresses as “Other”

In [8]:
customers_data.duplicated().sum()

np.int64(0)

In [9]:
customers_data['total_spend'].describe()

count    99032.000000
mean      4951.673197
std       1500.642398
min          0.000000
25%       3900.000000
50%       5000.000000
75%       6000.000000
max      11700.000000
Name: total_spend, dtype: float64

### Spending Analysis Overview

Based on the descriptive statistics of the `total_spend` column, we can observe several key insights:

1. **Data Quality Assessment**
   - No negative values detected in the spending data
   - All values are within reasonable business transaction ranges
   - The data distribution appears to be well-structured

2. **Key Metrics**
   - Mean customer spend: £4,951.67
   - Median customer spend: £5,000.00
   - Standard deviation: £1,500.64
   - Range: £0 to just under £12,000

3. **Distribution Characteristics**
   - 25th percentile: £3,900
   - 75th percentile: £6,000
   - The distribution shows a relatively normal pattern with slight right skew

This analysis provides a solid foundation for our geographic spending analysis.Now that we have seen our data, we need to decide on an approach to extract the information about cities.

In [10]:
#Before deciding on a method, we should look at some sample addresses.
for address in customers_data['address'].head(3):
    print(address,"\n")
for address in customers_data['address'].tail(3):
    print(address, "\n")

APARTMENT 2,
52 BEDFORD ROAD,
LONDON,
ENGLAND,
SW4 7HJ 

107 SHERINGHAM AVENUE,
LONDON,
N14 4UJ 

43 SUNNINGDALE,
YATE,
BRISTOL,
ENGLAND,
BS37 4HZ 

MARLAND HOUSE,
13 HUDDERSFIELD ROAD,
BARNSLEY,
SOUTH YORKSHIRE,
ENGLAND,
S70 2LW 

4 MOUNT SCAR VIEW,
SCHOLES,
HOLMFIRTH HUDDERSFIELD,
WEST YORKSHIRE,
HD9 1XH 

Manningham Mills Community Center, Silk Warehouse,
Lilycroft Road,
Bradford,
United Kingdom,
Bd9 5Be 



### Address Pattern Analysis

Initial examination of the sample addresses reveals several important patterns and considerations for data processing:

1. **Address Structure Patterns**
   - All addresses end with a postcode
   - Inconsistent inclusion of "ENGLAND" before postcodes
   - Address components are separated by commas and newline characters
   - Current sample shows uppercase formatting

2. **Data Quality Considerations**
   - Cannot rely on fixed position for city extraction
   - Need to implement robust parsing logic
   - Must handle inconsistent formatting

3. **Data Processing Strategy**
   - Standardize all addresses to uppercase format
   - Preserve original data in a separate column
   - Implement flexible parsing to handle variations

4. **Best Practices**
   - Maintain raw data for reference and validation
   - Create new column for processed addresses
   - Document any data transformations

This analysis will guide our approach to geographic data extraction and standardization.

In [11]:
customers_data['address_cleaned']=customers_data['address'].str.upper()
customers_data['address_lines']=customers_data['address_cleaned'].str.split(",\n").apply(len)
customers_data.head(3)

,company_id,address,total_spend,address_cleaned,address_lines
0,1,"APARTMENT 2,\n52 BEDFORD ROAD,\nLONDON,\nENGLA...",5700,"APARTMENT 2,\n52 BEDFORD ROAD,\nLONDON,\nENGLA...",5
1,2,"107 SHERINGHAM AVENUE,\nLONDON,\nN14 4UJ",4700,"107 SHERINGHAM AVENUE,\nLONDON,\nN14 4UJ",3
2,3,"43 SUNNINGDALE,\nYATE,\nBRISTOL,\nENGLAND,\nBS...",5900,"43 SUNNINGDALE,\nYATE,\nBRISTOL,\nENGLAND,\nBS...",5


In [12]:
customers_data['address_lines'].value_counts().sort_index()

address_lines
1        6
2       52
3     3284
4    35850
5    45931
6    13909
Name: count, dtype: int64

### Address Line Structure Analysis

Following the standardization of address casing, we conducted a comprehensive analysis of address line patterns:

1. **Address Line Distribution**
   - Total unique line count patterns: 6 different structures
   - Distribution breakdown:
     * 1-line addresses: 6 records
     * 2-line addresses: 52 records
     * 3-line addresses: 3,284 records
     * 4-line addresses: 35,850 records
     * 5-line addresses: 45,931 records
     * 6-line addresses: 13,909 records

2. **Key Observations**
   - 5-line addresses are the most common format (46.4% of records)
   - Significant variation in address structure complexity
   - Potential for pattern-based city extraction

3. **Analysis Methodology**
   - Split addresses into component lines using comma and newline delimiters
   - Calculated line count per address
   - Generated frequency distribution of address structures

4. **Next Steps**
   - Examine shorter address formats (1-2 lines) for potential patterns
   - Develop extraction rules based on line count patterns
   - Validate consistency within each address structure type



In [13]:
#Looks at six random rows that have only one line and two lines of address. 
print(customers_data.loc[customers_data["address_lines"] == 1, "address_cleaned"])
print(customers_data[(customers_data['address_lines'] == 1) | (customers_data['address_lines'] == 2)]
      .sample(6, random_state=5)['address_cleaned'])


17789                      FALKIRK
31897                   HADDINGTON
61750          CREAG BHAITHEACHAIN
75330                     NEWMILNS
78045    REDCLOAK FARM, STONEHAVEN
90897     REFER TO PARENT REGISTRY
Name: address_cleaned, dtype: object
57728                    1 RIDING HOUSE STREET,\nLONDON
78045                         REDCLOAK FARM, STONEHAVEN
62736    BRANCH REGISTRATION,\nREFER TO PARENT REGISTRY
90897                          REFER TO PARENT REGISTRY
97177                     50 JAMES STREET,\nHELENSBURGH
37527                          LANDS OF MUIRTON,\nPERTH
Name: address_cleaned, dtype: object


### City Extraction Strategy Analysis

**Current Data Challenges:**
- Some addresses contain only city names (e.g., "FALKIRK")
- Missing addresses with "REFER TO PARENT REGISTRY" notes
- Variable address line structures

**Potential Extraction Methods:**

1. **Line Position-Based**
   - Extract city based on fixed line position
   - Pros: Simple implementation
   - Cons: High error risk due to address variability

2. **City List Matching** (Recommended)
   - Match against predefined UK city list
   - Pros: Balance of accuracy and complexity
   - Cons: May miss non-city locations

3. **Postcode Database Lookup**
   - Use national postcode database
   - Pros: High accuracy
   - Cons: Requires external data source and permissions

4. **External API Integration**
   - Use services like Google Maps API
   - Pros: Comprehensive and accurate
   - Cons: Cost and API dependencies

**Implementation Plan:**
1. Source UK city list from [gov.uk](https://www.gov.uk/government/publications/list-of-cities/list-of-cities-html)
2. Convert to structured format (Excel/CSV)
3. Implement city name matching algorithm

In [14]:
cities = pd.read_csv(r"..\Data\cities.csv",header=None, names=["city"])
print(cities.head())
print(cities.shape)

              city
0          England
1             Bath
2      Birmingham*
3        Bradford*
4  Brighton & Hove
(80, 1)


There are clearly some aspects of this data to clean before using it as a definitive list of cities. First, the country headings were included as rows in the data, so the values England, Scotland, Wales, and Northern Ireland need to be removed. Then, the trailing asterisk * character also needs to be trimmed, and the remaining city names should be uppercased to match our address data.

In [15]:
countries_to_remove=['England','Scotland','Wales','Nothern Ireland']
index_to_remove=cities[cities['city'].isin(countries_to_remove)].index
cities=cities.drop(index=index_to_remove)
print(cities.shape)

cities['city']=cities['city'].str.replace('*','',regex=False)
cities['city']=cities['city'].str.upper()
cities.head()

(77, 1)


,city
1,BATH
2,BIRMINGHAM
3,BRADFORD
4,BRIGHTON & HOVE
5,BRISTOL


In [16]:
for city in cities["city"].values:
    customers_data.loc[customers_data["address_cleaned"].str.contains(f"\n{city},"),"city"] = city
    customers_data["city"] = customers_data["city"].fillna("OTHER")
customers_data['city'].value_counts()

city
OTHER                  54041
LONDON                 20762
MANCHESTER              1902
BIRMINGHAM              1866
GLASGOW                 1273
                       ...  
WESTMINSTER               10
ST ASAPH                   9
BRIGHTON & HOVE            2
NEWCASTLE-UPON-TYNE        2
ST DAVIDS                  2
Name: count, Length: 76, dtype: int64

Over half of our data is in the “Other” category, which means half of our customer base is established outside of major cities. This is an important insight to communicate to our stakeholders. Let’s look at some addresses from this category,

In [17]:
sample_of_other=customers_data[customers_data['city']=='OTHER'].sample(5,random_state=42)
for address in sample_of_other['address_cleaned'].values:
    print(address,'\n')

132 PENSBY ROAD,
HESWALL,
WIRRAL,
UNITED KINGDOM,
CH60 7RQ 

27 LONGMEAD,
HATFIELD,
AL10 0AD 

3 THE WEDGES,
ITCHINGFIELD,
HORSHAM,
ENGLAND,
RH13 0TA 

3 FILERS WAY,
WESTON GATEWAY BUSINESS PARK,
WESTON-SUPER-MARE,
UNITED KINGDOM,
BS24 7JP 

1 VERNON PLACE,
SCARBOROUGH,
ENGLAND,
YO11 2NH 



If we're looking for addresses specifically tied to London, the postcode would start with "E," "N," "NW," "SE," "SW," "W," or "WC," which are the main London postal districts.Unless we have knowledge of UK geography, we may miss these instances of London addresses without the word “London” in them.

#### Solution Approach
To address this issue, a solution that reclassifies addresses based on their postal codes:

1. **Postal Code Identification**: defined a list of postal code prefixes that are unique to London. This way, we can spot London addresses even if they don't say "London."

2. **Chunk Processing**: To efficiently handle the large dataset and avoid memory errors, we processed the data in smaller chunks. This keeps everything efficient and within limits.

3. **Reclassification Logic**: For each chunk, we extracted the postal code from the address and checked if it matched any of the London prefixes. If it did, we changed the city to "LONDON."

4. **Outcome**: This approach helped us reclassify a lot of addresses, giving us a clearer picture of our London customer base. Now, our city distribution is more accurate, which is great for planning. By using postal codes, we've made our geographic analysis more reliable, ensuring our business decisions are based on solid data.

In [18]:
# Define London postal district prefixes
london_districts = ['E', 'EC', 'N', 'NW', 'SE', 'SW', 'W', 'WC']

# Create a function to check if a postcode is in London
def is_london_postcode(postcode):
    for prefix in london_districts:
        if postcode.startswith(prefix) and (len(prefix) == 1 and postcode[1].isdigit() or len(prefix) == 2):
            return True
    return False

# Get indices of 'OTHER' addresses
other_indices = customers_data[customers_data['city'] == 'OTHER'].index

# Process in smaller chunks to avoid memory issues
chunk_size = 500
total_reclassified = 0

for i in range(0, len(other_indices), chunk_size):
    chunk_indices = other_indices[i:i + chunk_size]
    chunk_data = customers_data.loc[chunk_indices]
    
    # Extract postcodes and check if they're London postcodes
    postcodes = chunk_data['address_cleaned'].apply(lambda x: x.split('\n')[-1].strip())
    london_mask = postcodes.apply(is_london_postcode)
    
    # Update city for London addresses in this chunk
    customers_data.loc[chunk_indices[london_mask], 'city'] = 'LONDON'
    total_reclassified += london_mask.sum()

print("\nUpdated city distribution after London postcode correction:")
print(customers_data['city'].value_counts())
print(f"\nTotal addresses reclassified as London: {total_reclassified}")



Updated city distribution after London postcode correction:
city
OTHER                  53872
LONDON                 20931
MANCHESTER              1902
BIRMINGHAM              1866
GLASGOW                 1273
                       ...  
WESTMINSTER               10
ST ASAPH                   9
BRIGHTON & HOVE            2
NEWCASTLE-UPON-TYNE        2
ST DAVIDS                  2
Name: count, Length: 76, dtype: int64

Total addresses reclassified as London: 169


One sanity check at this point is to see how many unique cities are in the government data and our tagged address data. Depending on the business, we might assume that we have at least one customer in every major UK city, and we can verify this.

In [19]:
set(cities['city'])-set(customers_data['city'])

{'KINGSTON-UPON-HULL'}

The output of this code is the string {'KINGSTON-UPON-HULL'}, which tells us that the city Kingston-upon-Hull is not in the city column of our customer address data.This either means we have no customers there, which is not impossible given its population is only around 250,000, or there is something else going on. The city is usually abbreviated to just “Hull,” which again is a case of applying specific domain knowledge to the problem, so let’s look for that in our address data:

In [20]:
customers_data[customers_data['address_cleaned'].str.contains('\nHULL,')]

,company_id,address,total_spend,address_cleaned,address_lines,city
455,456,"349 BEVERLEY ROAD,\nANLABY,\nHULL,\nENGLAND,\n...",3800,"349 BEVERLEY ROAD,\nANLABY,\nHULL,\nENGLAND,\n...",5,OTHER
1139,1140,"90 CALVERT LANE,\nHULL,\nENGLAND,\nHU4 6BJ",6000,"90 CALVERT LANE,\nHULL,\nENGLAND,\nHU4 6BJ",4,OTHER
1822,1823,"ACER GLADE BLACK TUP LANE,\nARNOLD,\nHULL,\nEN...",4600,"ACER GLADE BLACK TUP LANE,\nARNOLD,\nHULL,\nEN...",5,OTHER
1831,1832,"10 GORDON STREET,\nHULL,\nENGLAND,\nHU3 3HN",5300,"10 GORDON STREET,\nHULL,\nENGLAND,\nHU3 3HN",4,OTHER
1921,1922,"70 WRIGHT STREET,\nHULL,\nENGLAND,\nHU2 8JD",6100,"70 WRIGHT STREET,\nHULL,\nENGLAND,\nHU2 8JD",4,OTHER
...,...,...,...,...,...,...
98479,98480,"31-38 QUEEN STREET,\nHULL,\nENGLAND,\nHU1 1UU",4500,"31-38 QUEEN STREET,\nHULL,\nENGLAND,\nHU1 1UU",4,OTHER
98886,98887,"13 SALMON GROVE,\nHULL,\nEAST YORKSHIRE,\nENGL...",5000,"13 SALMON GROVE,\nHULL,\nEAST YORKSHIRE,\nENGL...",5,OTHER
99196,99197,"22 19TH AVENUE,\nHULL,\nENGLAND,\nHU6 8HF",5900,"22 19TH AVENUE,\nHULL,\nENGLAND,\nHU6 8HF",4,OTHER
99434,99435,"F14 THE BLOC SPRINGFIELD WAY,\nANLABY,\nHULL,\...",3100,"F14 THE BLOC SPRINGFIELD WAY,\nANLABY,\nHULL,\...",6,OTHER


The output of this code tells us there are 284 relevant records. We can manually update our city column to fix this problem:

In [21]:
customers_data.loc[customers_data['address_cleaned'].str.contains('\nHULL'),'city']='HULL'
print(customers_data['city'].value_counts())

city
OTHER                  53578
LONDON                 20931
MANCHESTER              1902
BIRMINGHAM              1866
GLASGOW                 1273
                       ...  
WESTMINSTER               10
ST ASAPH                   9
BRIGHTON & HOVE            2
NEWCASTLE-UPON-TYNE        2
ST DAVIDS                  2
Name: count, Length: 77, dtype: int64


In [22]:
customers_data.to_csv(r"..\Data\cleaned_customers_data.csv")

In [24]:
customers_data.head()

,company_id,address,total_spend,address_cleaned,address_lines,city
0,1,"APARTMENT 2,\n52 BEDFORD ROAD,\nLONDON,\nENGLA...",5700,"APARTMENT 2,\n52 BEDFORD ROAD,\nLONDON,\nENGLA...",5,LONDON
1,2,"107 SHERINGHAM AVENUE,\nLONDON,\nN14 4UJ",4700,"107 SHERINGHAM AVENUE,\nLONDON,\nN14 4UJ",3,LONDON
2,3,"43 SUNNINGDALE,\nYATE,\nBRISTOL,\nENGLAND,\nBS...",5900,"43 SUNNINGDALE,\nYATE,\nBRISTOL,\nENGLAND,\nBS...",5,BRISTOL
3,4,"HAWESWATER HOUSE,\nLINGLEY MERE BUSINESS PARK,...",7200,"HAWESWATER HOUSE,\nLINGLEY MERE BUSINESS PARK,...",5,OTHER
4,5,"AMBERFIELD BARN HOUSE AMBER LANE,\nCHART SUTTO...",4600,"AMBERFIELD BARN HOUSE AMBER LANE,\nCHART SUTTO...",5,OTHER


In [25]:
# Aggregate total spend by city
regional_spending = customers_data.groupby('city')['total_spend'].sum().reset_index()

# Display the result
print(regional_spending)

             city  total_spend
0        ABERDEEN      1322700
1          ARMAGH       263400
2          BANGOR       292000
3            BATH      1010400
4         BELFAST      2059400
..            ...          ...
72     WINCHESTER       613900
73  WOLVERHAMPTON      1907600
74      WORCESTER       867900
75        WREXHAM       580800
76           YORK      1728500

[77 rows x 2 columns]


In [28]:
regional_spending.to_csv(r'..\Data\RegionalSpending.csv')

In [29]:
# Calculate total spend
total_spend = customers_data['total_spend'].sum()

# Calculate market penetration
customers_data['market_penetration'] = (customers_data['total_spend'] / total_spend) * 100

# Aggregate by city
city_market_penetration = customers_data.groupby('city')['market_penetration'].sum().reset_index()
city_market_penetration.to_csv(r'..\Data\CityMarketPenetration.csv')
# Display the result
print(city_market_penetration)

             city  market_penetration
0        ABERDEEN            0.269733
1          ARMAGH            0.053714
2          BANGOR            0.059546
3            BATH            0.206047
4         BELFAST            0.419965
..            ...                 ...
72     WINCHESTER            0.125190
73  WOLVERHAMPTON            0.389009
74      WORCESTER            0.176987
75        WREXHAM            0.118440
76           YORK            0.352486

[77 rows x 2 columns]
